<a href="https://colab.research.google.com/github/kishorex18/PytorchDL/blob/main/BidirectionalRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
batch_size=64
embedding_dim=28
sequence_length=28
learning_rate=0.001
epochs=3
hidden_size=256
num_classes=10
num_layers=2

In [3]:
train_data=MNIST(root="dataset/",train=True,transform=transforms.ToTensor(),download=True)
train_batches=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_data=MNIST(root="dataset/",train=False,transform=transforms.ToTensor(),download=True)
test_batches=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:11<00:00, 900kB/s] 


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:06<00:00, 246kB/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.11MB/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [5]:
import torch.cuda
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
torch.cuda.is_available()

True

In [7]:
class BLSTM(nn.Module):
  def __init__(self,input_size,num_layers,hidden_size):
    super(BLSTM,self).__init__()
    self.input_size=input_size
    self.num_layers=num_layers
    self.hidden_size=hidden_size

    self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
    self.fc=nn.Linear(2*hidden_size,num_classes)

  def forward(self,X):

      h0=torch.zeros(2*num_layers,X.size(0),self.hidden_size).to(device)
      c0=torch.zeros(2*num_layers,X.size(0),self.hidden_size).to(device)
      output,_=self.lstm(X,(h0,c0))
      output=self.fc(output[:,-1,:]) #processing only the final output
      return output

In [8]:
def train(model,train_batches):
    creterion=nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=learning_rate)
    for i in range(epochs):
      for batch_idx,(data,targets) in enumerate(train_batches):
        data=data.to(device).squeeze(1)
        targets=targets.to(device)
        predicted=model(data)
        loss=creterion(predicted,targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    return model

In [9]:
def check_accuracy(batches,model):
  num_samples=0
  num_correct=0
  model.eval()
  with torch.no_grad():
    for data,targets in batches:
      data=data.to(device).squeeze(1)
      targets=targets.to(device)
      output=model(data)
      _,predicted=output.max(1)
      num_correct+=(targets==predicted).sum()
      num_samples+=predicted.size(0)

  model.train()
  return num_correct/num_samples

In [12]:
model_blstm=BLSTM(embedding_dim,num_layers,hidden_size).to(device)

In [13]:
trained_model_blstm=train(model_blstm,train_batches)

In [14]:
print(check_accuracy(test_batches,model_blstm))

tensor(0.9788, device='cuda:0')
